In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  
import tensorflow as tf

k = tf.keras

In [2]:
with open('./shakespeare.txt') as f:
    txt = f.read()
print(txt[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [3]:
chars = sorted(set(txt))

In [4]:
char_to_ind = dict(zip(chars, range(len(chars))))

In [5]:
ind_to_char = chars

In [6]:
encoded_txt = [char_to_ind[i] for i in txt]

In [7]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_txt)

In [8]:
sequences = char_dataset.batch(128, drop_remainder=True)

In [9]:
def create_seq_targets(seq):
    input_seq = seq[:-1]
    target_seq = seq[1:]
    return input_seq, target_seq

In [10]:
dataset = sequences.map(create_seq_targets)

In [11]:
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size, seed=101).batch(batch_size, drop_remainder=True)

In [12]:
def create_model(batch_size):
    model = k.Sequential()
    model.add(k.layers.Embedding(len(chars), 64, batch_input_shape=(batch_size, None)))
    model.add(k.layers.GRU(1026, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(k.layers.Dense(len(chars)))
    model.compile(optimizer='adam', loss=k.losses.SparseCategoricalCrossentropy(from_logits=True))
    
    return model

In [13]:
model = create_model(batch_size)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [15]:
model = create_model(batch_size=1)

In [16]:
model.load_weights('./shakespeare_gen.h5')

In [17]:
model.build((1, None))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [19]:
def generate_text(model, start_seed, gen_size):
    text_generated = []
    input_eval = [char_to_ind[i] for i in start_seed]
    input_eval = tf.expand_dims(input_eval, axis=0)
    model.reset_states()
    
    for i in range(gen_size):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [20]:
print(generate_text(model, 'Romeo And Juliet', 1000))

Romeo And Juliet else himself between 'a may preferment
    under him; overfare sheep. Do you know what?
  SHYLOCK. Offeron, men and has directed wherce,
    Your Vicoors?
  POLIXENES. Sooth, sweet madam;
    These e. Where sent to the last, that it is so?
    How came it to the King? O you bury!
    I tell your good Crnnnian mild with kind;
    Except this habiliman,
    That my Nyr spark how much of flaw?
  PAULINA. The care on question;
    Which if his purposes it is sweet messes, to make appear,
    Your Valentine is writ down on our charge.
    Part Love can walk again, good Caesar-
  ANTONIO. O, wife.
  DESDEMONA. Good night, good warron--should be that the parageren wear commonwealth be silent,  
    Nay, I'll be, lov'd, have pardon'd your son, to be scoutable;
    For ere the greater banish'd throne in's name,
    Liegemonously bestrew.
  STEPHERD. Nothing.
  RODO. Even so much faith and brought you on your  hearted serving,
    So tore, as our friends kill all have, and there